In [2]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, AutoConfig
from peft import LoraConfig, get_peft_model
import numpy as np
import random
from torch.nn import Module, Parameter, ParameterList
from typing import List

/home/hieupq1/.conda/envs/llama_fac/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No CUDA runtime is found, using CUDA_HOME='/home/hieupq1/.conda/envs/llama_fac'


In [3]:
mask = torch.LongTensor([[1, 1, 1, 1]])
mask[:, None, None, :]

tensor([[[[1, 1, 1, 1]]]])

In [5]:
class Test():
    def __init__(self, name="Hieu", **kwargs):
        self.name = name
        print(kwargs)

    def say_hi(self, name="Pham"):
        print("hi", name)

def say_hello(self, name):
    print("hello", name)

a = Test(**{"name_2": "Pham"})
setattr(a, "say_hi", say_hello)
a.say_hi()

{'name_2': 'Pham'}


TypeError: say_hello() missing 1 required positional argument: 'self'

In [2]:
class CustomEmbedding(Module):
    """Custom embedding layers for training with reserved tokens only"""
    def __init__(self, original_embeddings, token_ids: List):
        super(CustomEmbedding, self).__init__()
        self.original_embeddings = original_embeddings
        self.token_ids = token_ids
        self.extra_token_embeddings = ParameterList([Parameter(data=original_embeddings.weight[token_id].clone(), requires_grad=True) for token_id in self.token_ids])

        self.id_map = {}
        for index, token_id in enumerate(self.token_ids):
            self.id_map[token_id] = index

        # freeze original embedding tokens
        self.original_embeddings.weight.requires_grad = False

    def forward(self, input_ids):
        embeddings = self.original_embeddings(input_ids).detach()
        for row_id in range(input_ids.shape[0]):
            for pos, token_id in enumerate(input_ids[row_id]):
                token_id_cpu = token_id.cpu().item()

                if token_id_cpu in self.token_ids:
                    embeddings[row_id, pos, :] = self.extra_token_embeddings[self.id_map[token_id_cpu]]
        return embeddings

def extend_vocab(model, tokenizer):
    """
        Replace the model Embedding with the custom embedding to learn newly added tokens only
    """
    num_new_token = 100 # default value

    original_embedding = model.get_input_embeddings() # to get the embedding layer

    original_embedding_vocab_size = original_embedding.weight.shape[0]
    original_vocab_size = len(tokenizer)

    num_reserved_tokens = original_embedding_vocab_size - original_vocab_size

    # add new token 
    num_new_token = max(num_new_token, num_reserved_tokens)

    tokenizer.add_tokens([f"<sentinel_tok_{i}>" for i in range(num_new_token)])
    
    # print(tokenizer.encode("<sentinel_tok_10>"))
    new_embedding = CustomEmbedding(model.resize_token_embeddings(len(tokenizer)), [original_vocab_size+i for i in range(num_new_token)])

    model.set_input_embeddings(new_embedding)
    
    return model, tokenizer


In [3]:
# accelerator = Accelerator()
print("START LOADING MODEL")
lora_path = "/home/hieupq1/hieupq1/math/saves/deepseek-math-ul2-debug/checkpoint-232"
state_dict_path = f"{lora_path}/global_step{lora_path.split('-')[-1]}/mp_rank_00_model_states.pt"
base_path = "/home/hieupq1/hieupq1/math/deepseek-math-7b-instruct/"

tokenizer = AutoTokenizer.from_pretrained(base_path)
print(len(tokenizer))

state_dict = torch.load(state_dict_path)["module"]

def get_extended_embeddings_state_dict(state_dict, extra_param_name="extra_token_embeddings"):
    # load embedding layer
    result = {}
    for key in state_dict.keys():
        if extra_param_name in key:
            tmp = key.split(".")
            param_name = f"{tmp[-2]}.{tmp[-1]}"

            result[param_name] = state_dict[key]
    return result


lora_config = LoraConfig.from_pretrained(lora_path)

model = AutoModelForCausalLM.from_pretrained(base_path)
model = get_peft_model(model, lora_config)
model, tokenizer = extend_vocab(model, tokenizer)


print(model.__dir__)

model.from_pretrained(model, model_id=lora_path)
embedding = model.get_input_embeddings()
embedding.load_state_dict(get_extended_embeddings_state_dict(state_dict), strict=False)
# for name, parameter in embedding.named_parameters():

print(model.base_model.model.model.embed_tokens.extra_token_embeddings[9])


START LOADING MODEL


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


100002


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]


<bound method Module.__dir__ of PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): CustomEmbedding(
          (original_embeddings): Embedding(102400, 4096)
          (extra_token_embeddings): ParameterList(
              (0): Parameter containing: [torch.float32 of size 4096]
              (1): Parameter containing: [torch.float32 of size 4096]
              (2): Parameter containing: [torch.float32 of size 4096]
              (3): Parameter containing: [torch.float32 of size 4096]
              (4): Parameter containing: [torch.float32 of size 4096]
              (5): Parameter containing: [torch.float32 of size 4096]
              (6): Parameter containing: [torch.float32 of size 4096]
              (7): Parameter containing: [torch.float32 of size 4096]
              (8): Parameter containing: [torch.float32 of size 4096]
              (9): Parameter containing: [torch.float32 of size 4096]
            

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
tokenizer.decode(100001)

'<｜end▁of▁sentence｜>'

Parameter containing:
tensor([-0.1011, -0.2734,  0.1914,  ...,  0.2041, -0.0488,  0.0986])


In [10]:
print(model.base_model.model.model.embed_tokens(torch.LongTensor([[9999]])))

tensor([[[ 0.2041,  0.0420,  0.2451,  ..., -0.0214,  0.2793,  0.0198]]])


In [11]:
model.base_model.model.model.embed_tokens.original_embeddings(torch.LongTensor([[9999]]))

tensor([[[ 0.2041,  0.0420,  0.2451,  ..., -0.0214,  0.2793,  0.0198]]])

In [17]:
state_dict["module"]["base_model.model.model.embed_tokens.extra_token_embeddings.8"]

tensor([ 1.7480e-01, -1.2398e-04,  6.9336e-02,  ..., -3.1494e-02,
        -2.5781e-01,  2.1387e-01], device='cuda:0', dtype=torch.bfloat16)

In [7]:
state_dict = torch.load("/home/hieupq1/hieupq1/math/saves/deepseek-math-ul2-debug/checkpoint-232/global_step232/mp_rank_00_model_states.pt")

In [8]:
state_dict["module"]

OrderedDict([('base_model.model.model.embed_tokens.extra_token_embeddings.0',
              tensor([ 0.0097, -0.1865, -0.0006,  ..., -0.1426, -0.3809,  0.0649],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.embed_tokens.extra_token_embeddings.1',
              tensor([-0.1133, -0.2617,  0.2891,  ..., -0.2988, -0.1387,  0.2188],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.embed_tokens.extra_token_embeddings.2',
              tensor([-0.1807, -0.0552,  0.2793,  ..., -0.0265, -0.0996, -0.1133],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.embed_tokens.extra_token_embeddings.3',
              tensor([-0.1426, -0.1289,  0.0464,  ...,  0.0339,  0.0752,  0.0928],
                     device='cuda:0', dtype=torch.bfloat16)),
             ('base_model.model.model.embed_tokens.extra_token_embeddings.4',
              tensor([-0.2578,

In [2]:
state = torch.load("/home/hieupq1/hieupq1/math/saves/deepseek-math-ul2-debug/checkpoint-116/global_step116/mp_rank_00_model_states.pt")

In [6]:
state["module"].keys()

odict_keys(['base_model.model.model.embed_tokens.extra_token_embeddings.0', 'base_model.model.model.embed_tokens.extra_token_embeddings.1', 'base_model.model.model.embed_tokens.extra_token_embeddings.2', 'base_model.model.model.embed_tokens.extra_token_embeddings.3', 'base_model.model.model.embed_tokens.extra_token_embeddings.4', 'base_model.model.model.embed_tokens.extra_token_embeddings.5', 'base_model.model.model.embed_tokens.extra_token_embeddings.6', 'base_model.model.model.embed_tokens.extra_token_embeddings.7', 'base_model.model.model.embed_tokens.extra_token_embeddings.8', 'base_model.model.model.embed_tokens.extra_token_embeddings.9', 'base_model.model.model.embed_tokens.extra_token_embeddings.10', 'base_model.model.model.embed_tokens.extra_token_embeddings.11', 'base_model.model.model.embed_tokens.extra_token_embeddings.12', 'base_model.model.model.embed_tokens.extra_token_embeddings.13', 'base_model.model.model.embed_tokens.extra_token_embeddings.14', 'base_model.model.model

In [2]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-math-7b-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
def _mask_segments(arr, n, mu, distribution='normal', tokenizer=None):
    if distribution not in ['normal', 'uniform']:
        raise ValueError("Distribution must be 'normal' or 'uniform'")
    
    masked_solution = []
    denoised_solution = []

    length = len(arr)
    masked_indices = set()

    for _ in range(n):
        # Generate a segment length based on the chosen distribution
        if distribution == 'normal':
            seg_length = int(np.abs(np.random.normal(mu)))
        else:
            seg_length = int(np.random.uniform(0, 2*mu))
        
        # Ensure segment length is at least 1
        seg_length = max(1, seg_length)
        
        # Find a valid start position for the segment
        while True:
            start = random.randint(0, length - seg_length)
            segment = set(range(start, start + seg_length))

            if not segment & masked_indices:  # Ensure no overlap
                break
        
        # Add the segment indices to the masked_indices set
        masked_indices.update(segment)
    
    # Create the masked list
    # TODO: finish this part to mimic denoising setting 
    sentinel_num = 0
    
    for i in range(length):
        if i in masked_indices:
            if i-1 > 0 and i-1 not in masked_indices:
                sentinel_tok = tokenizer.encode(f"<sentinel_tok_{sentinel_num}>", add_special_tokens=False)
                masked_solution.extend(sentinel_tok)
                denoised_solution.extend(sentinel_tok)
                sentinel_num += 1

            denoised_solution.append(arr[i])
        else:
            masked_solution.append(arr[i])

    # masked_arr = [mask_value if i in masked_indices else arr[i] for i in range(length)]
    # masked_solution.extend(denoised_solution)
    result = []
    result.extend(masked_solution)
    result.extend(denoised_solution)

    return result, len(masked_solution)


In [6]:
x = [1]

In [8]:
x

[1, 1]

In [3]:
tokenizer.add_tokens([f"<sentinel_tok_{i}>" for i in range(100)])

100

In [9]:
tokenizer.decode([27, 17590, 37891, 62, 26491, 62, 15, 29, 11, 279, 207, 16, 20, 1555, 11, 838, 6, 647, 330, 207, 18, 15, 10, 16, 20, 28, 12216, 18, 15, 10, 16, 20, 28, 19, 20, 6979, 19, 20, 1555, 1712, 13, 185, 3567, 207, 18, 15, 11, 1640, 18358, 317, 207, 19, 14, 18, 254, 4500, 280, 52321, 522, 11, 4569, 52321, 522, 317, 207, 19, 14, 18, 9, 18, 15, 28, 12216, 19, 14, 18, 9, 18, 15, 28, 19, 15, 6979, 19, 15, 1555, 13, 185, 774, 207, 16, 20, 1555, 11, 52321, 522, 543, 330, 207, 19, 15, 10, 16, 20, 28, 12216,
 19, 15, 10, 16, 20, 28, 20, 20, 6979, 20, 20, 1555, 1712, 13, 185, 24925, 3223, 4500, 279, 207, 16, 20, 1555, 543, 330, 207, 20, 20, 10, 19, 20, 28, 12216, 20, 20, 10, 19, 20, 28, 16, 15, 15, 6979, 16
, 15, 15, 185, 24925, 5137, 4500, 279, 207, 16, 20, 1555, 543, 330, 207, 16, 15, 15, 14, 17, 28, 12216, 16, 15, 15, 14, 17, 28, 20, 15, 6979, 20, 15, 185, 3589, 207, 20, 15, 100001])

"sentinel_tok_0>, in 15 years, she'll be 30+15=<<30+15=45>>45 years old.\nAt 30, Anika is 4/3 the age of Maddie, meaning Maddie is 4/3*30=<<4/3*30=40>>40 years.\nIn 15 years, Maddie will be 40+15=<<40+15=55>>55 years old.\nTheir total age in 15 years will be 55+45=<<55+45=100>>100\nTheir average age in 15 years will be 100/2=<<100/2=50>>50\n#### 50<｜end▁of▁sentence｜>"

In [17]:
seq = tokenizer.encode("Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72", add_special_tokens=False)

In [22]:
masked_arr, prefix_len = _mask_segments(seq, n=int(0.15*len(seq)), mu=3, tokenizer=tokenizer)

In [24]:
masked_arr

[87499,
 480,
 5151,
 207,
 19,
 23,
 14,
 17,
 403,
 6523,
 19,
 100002,
 17,
 28,
 100003,
 17,
 19,
 34406,
 279,
 3638,
 100004,
 5151,
 207,
 19,
 23,
 100005,
 19,
 100006,
 23,
 10,
 17,
 19,
 28,
 100007,
 16369,
 100008,
 3638,
 13,
 1501,
 13483,
 207,
 22,
 17,
 100002,
 23,
 14,
 100003,
 17,
 19,
 6979,
 100004,
 13,
 39203,
 480,
 100005,
 10,
 17,
 100006,
 403,
 6523,
 19,
 100007,
 22,
 17,
 6979,
 22,
 17,
 34406,
 100008,
 279,
 6511,
 285]

In [25]:
tokenizer.decode(masked_arr)

'Natalia sold 48/2 = <<4<sentinel_tok_0>2=<sentinel_tok_1>24 clips in May<sentinel_tok_2> sold 48<sentinel_tok_3>4<sentinel_tok_4>8+24=<sentinel_tok_5> altogether<sentinel_tok_6> May. #### 72<sentinel_tok_0>8/<sentinel_tok_1>24>><sentinel_tok_2>. Natalia<sentinel_tok_3>+2<sentinel_tok_4> = <<4<sentinel_tok_5>72>>72 clips<sentinel_tok_6> in April and'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/lustre/scratch/client/vinai/users/hieupq1/huggingface/hub/deepseek-math-7b-instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
model = AutoModel.from_pretrained("deepseek-math-7b-instruct", device_map="cpu")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer.apply_chat_template([{"role": "user", "content": "hello my name is"}, {"role": "assistant", "content": "dog"}, {"role": "user", "content": "hello my name is"}], tokenize=False)

'<｜begin▁of▁sentence｜>User: hello my name is\n\nAssistant: dog<｜end▁of▁sentence｜>User: hello my name is\n\n'

In [12]:
tokenizer.bos_token == "<｜begin▁of▁sentence｜>"

True

In [13]:
input_ids = [[1, 2, 3], [4, 5, 6]]
[[1] * len(input_ids[0])] * len(input_ids)

[[1, 1, 1], [1, 1, 1]]

In [11]:
import numpy as np
import random

def _shuffle_segments(arr, n, mu, distribution='normal'):
    if distribution not in ['normal', 'uniform']:
        raise ValueError("Distribution must be 'normal' or 'uniform'")
    
    length = len(arr)
    shuffled_arr = arr.copy()
    shuffled_indices = set()

    for _ in range(n):
        # Generate a segment length based on the chosen distribution
        if distribution == 'normal':
            seg_length = int(np.abs(np.random.normal(mu)))
        else:
            seg_length = int(np.random.uniform(0, 2*mu))
        
        # Ensure segment length is at least 1 and does not exceed the array length
        seg_length = max(1, min(seg_length, length))
        
        # Find a valid start position for the segment
        while True:
            start = random.randint(0, length - seg_length)
            segment = set(range(start, start + seg_length))
            if not segment & shuffled_indices:  # Ensure no overlap
                break
        
        # Add the segment indices to the shuffled_indices set
        shuffled_indices.update(segment)
        
        # Shuffle the segment within the array
        segment_list = list(segment)
        segment_values = [shuffled_arr[i] for i in segment_list]
        random.shuffle(segment_values)
        for i, idx in enumerate(segment_list):
            shuffled_arr[idx] = segment_values[i]

    return shuffled_arr, shuffled_indices

# Example usage
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
n = 2
mu = 3
shuffled_arr = _shuffle_segments(arr, n, mu, distribution='normal')
print(shuffled_arr)


([1, 2, 3, 5, 4, 6, 8, 7, 9, 10], {3, 4, 5, 6, 7, 8})


In [6]:
tokenizer.mask_token_id

In [3]:
tmp1 = [[1, 2]]
tmp2= [[3, 4], [5, 6]]
tmp1.extend(tmp2)

In [4]:
tmp1

[[1, 2], [3, 4], [5, 6]]

In [10]:
import random
import numpy as np

def mask_segments(arr, n, mu, distribution='normal', mask_value=-100):
    if distribution not in ['normal', 'uniform']:
        raise ValueError("Distribution must be 'normal' or 'uniform'")
    
    length = len(arr)
    masked_indices = set()

    for _ in range(n):
        # Generate a segment length based on the chosen distribution
        if distribution == 'normal':
            seg_length = int(np.abs(np.random.normal(mu)))
        else:
            seg_length = int(np.random.uniform(0, 2*mu))
        
        # Ensure segment length is at least 1
        seg_length = max(1, seg_length)
        
        # Find a valid start position for the segment
        while True:
            start = random.randint(0, length - seg_length)
            segment = set(range(start, start + seg_length))
            if not segment & masked_indices:  # Ensure no overlap
                break
        
        # Add the segment indices to the masked_indices set
        masked_indices.update(segment)
    
    # Create the masked list
    masked_arr = [mask_value if i in masked_indices else arr[i] for i in range(length)]
    return masked_arr

# Example usage
arr = list(range(100))
n = 5
mu = 5
masked_arr = mask_segments(arr, n, mu, distribution='normal', mask_value=-100)
print(masked_arr)


-100
[0, 1, 2, 3, 4, 5, 6, 7, 8, -100, -100, -100, -100, -100, 14, 15, 16, 17, 18, 19, 20, 21, -100, -100, -100, -100, -100, -100, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, -100, -100, -100, -100, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, -100, -100, -100, -100, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, -100, -100, -100, -100, -100, 94, 95, 96, 97, 98, 99]


In [20]:
def prepare_4d_attention_mask(attention_mask_with_indices: "torch.Tensor", prefix_lengths: "torch.Tensor", dtype: "torch.dtype") -> "torch.Tensor":
    r"""
    Expands the attention mask with indices from (batch_size, seq_len) to (batch_size, 1, seq_len, seq_len),
    while handles packed sequences and transforms the mask to lower triangular form to prevent future peeking.

    e.g.
    ```python
    # input
    [[1, 1, 2, 2, 2, 0]]
    # output
    [
        [
            [
                [o, x, x, x, x, x],
                [o, o, x, x, x, x],
                [x, x, o, x, x, x],
                [x, x, o, o, x, x],
                [x, x, o, o, o, x],
                [x, x, x, x, x, x],
            ]
        ]
    ]
    ```
    where `o` equals to `0.0`, `x` equals to `min_dtype`.
    """
    bsz, seq_len = attention_mask_with_indices.size()
    min_dtype = torch.finfo(dtype).min
    expanded_mask = attention_mask_with_indices[:, None, None, :].expand(bsz, 1, seq_len, seq_len)
    print("expanded mask", expanded_mask)
    # Create a binary mask from the original mask where zeros remain zeros and all other values are set to one
    padding_mask = torch.where(expanded_mask != 0, 1, 0)
    # Create a block-diagonal mask.
    attention_mask_4d = torch.eq(expanded_mask, expanded_mask.transpose(-1, -2)).int() * padding_mask
    
    # Use the lower triangular mask to zero out the upper triangular part
    attention_mask_4d *= torch.tril(torch.ones((seq_len, seq_len), dtype=torch.long))
    
    for batch in range(attention_mask_4d.shape[0]):
        for i in range(attention_mask_4d.shape[-1]):
            if attention_mask_4d[batch, :, i, i] == 1:
                break
        # print(batch, i, prefix_lengths[batch].item())
        prefix_len = int(prefix_lengths[batch].item())
        attention_mask_4d[batch, :, i:i+prefix_len, i:i+prefix_len] = 1.0

    print('attention_mask_4d', attention_mask_4d)

    # Invert the attention mask.
    attention_mask_4d = torch.where(attention_mask_4d != 0, torch.tensor(0, dtype=dtype), min_dtype)
    return attention_mask_4d


In [21]:
prepare_4d_attention_mask(torch.Tensor([[ 0, 0, 0, 1, 1 ,1, 1, 1], [0, 0, 1, 1 ,1, 1, 1, 1]]), torch.Tensor([2, 2]),dtype=torch.float16)

expanded mask tensor([[[[0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.]]],


        [[[0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1., 1.]]]])
attention_mask_4d tensor([[[[0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 1, 0, 0, 0],
          [0, 0, 0, 1, 1, 0, 0, 0],
          [0, 0, 0, 1, 1, 1, 0, 0],
          [0, 0, 0, 1, 1, 1, 1, 0],
     

tensor([[[[-65504., -65504., -65504., -65504., -65504., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504., -65504., -65504., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504., -65504., -65504., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504.,      0.,      0., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504.,      0.,      0., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504.,      0.,      0.,      0., -65504.,
           -65504.],
          [-65504., -65504., -65504.,      0.,      0.,      0.,      0.,
           -65504.],
          [-65504., -65504., -65504.,      0.,      0.,      0.,      0.,
                0.]]],


        [[[-65504., -65504., -65504., -65504., -65504., -65504., -65504.,
           -65504.],
          [-65504., -65504., -65504., -65504., -65504., -65504., -65504.,
           -65504.],
          [-65504., -65504.,      0.,      0.,